In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-06-25 17:57:30 __main__ INFO     torch.__version__='2.5.1+cu121', torch.version.cuda='12.1'
2025-06-25 17:57:30 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2025-06-25 17:57:30 __main__ INFO     transformers.__version__='4.52.4'


In [3]:
from src.utils.training_utils import get_device_map

model_key = "meta-llama/Llama-3.2-1B-Instruct"

device_map = get_device_map(model_key, 32, 1)
device_map

2025-06-25 17:57:30 src.utils.training_utils WARNING   Model meta-llama/Llama-3.2-1B-Instruct not supported  Only 1 GPU(s) available using default device map = `auto`.


'auto'

In [4]:
from src.models import ModelandTokenizer

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    device_map=device_map
)

2025-06-25 17:57:33 src.models WARNING  meta-llama/Llama-3.2-1B-Instruct not found in data/hf_cache
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-06-25 17:57:33 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-06-25 17:57:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-06-25 17:57:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.2-1B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-06-25 17:57:34 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.2-1B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
2025-06-25 17:57:34 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set 

In [5]:
#from typing import Literal
#from src.probing.prompt import ProbingPrompt
#from src.hooking.llama_attention import AttentionEdge
#
#def get_edges_to_be_ablated(
#    prompt: ProbingPrompt,
#    focus_strategy: Literal["entities", "entities_last", "ablate_all"] = "ablate_all",
#    Q_IDX: int = -1, # almost always the last token position
#    whitelist_key_indices: list[int] = [0, -1],
#):
#    """
#    For attention ablation experiments.
#    Determines which attention connections should be blocked (ablated) between tokens.
#    Used to understand which attention patterns are important for specific model behaviors.
#
#    Creates a block list of attention edges by:
#        1. Converting negative indices to positive ones
#        2. Building a whitelist of key positions to preserve based on the strategy
#        3. Creating AttentionEdge objects for all non-whitelisted positions
#
#    Whitelisting strategy: Instead of specifying which edge to ablate, specify which connections
#        to preserve, and everything else gets ablated.
#
#    Arguments:
#        focus_strategy: controls which entity-related tokens to preserve
#            entities: keeps all tokens within both entity ranges
#            entities_last: only keeps the last token of each entity
#            ablate_all: no special entity preservation
#        Q_IDX: the query token position (usually the last token where we want to measure the effect)
#        whitelist_key_indices: additional key positions to preserve (defaults to first and last tokens)
#
#    Returns:
#        AttentionEdge: represent specific attention connections from a query token position
#            to key token positions that should be blocked.
#    """
#    for idx, ti in enumerate(whitelist_key_indices):
#        if ti < 0:
#            whitelist_key_indices[idx] = prompt.tokenized["input_ids"][0].shape[-1] + ti
#    
#    if focus_strategy == "entities":
#        whitelist_key_indices += list(range(*prompt.entity_ranges[0])) + list(
#            range(*prompt.entity_ranges[1])
#        )
#    elif focus_strategy == "entities_last":
#        whitelist_key_indices += [
#            prompt.entity_ranges[0][1] - 1,
#            prompt.entity_ranges[1][1] - 1,
#        ]
#    elif focus_strategy == "ablate_all":
#        pass
#    else:
#        raise ValueError(f"{focus_strategy=}")
#
#    whitelist_key_indices = list(set(whitelist_key_indices))
#    if Q_IDX < 0:
#        Q_IDX = prompt.tokenized["input_ids"][0].shape[-1] + Q_IDX
#
#    block_edges: list[AttentionEdge] = []
#    for k_idx in range(0, prompt.tokenized["input_ids"][0].shape[-1]):
#        if k_idx in whitelist_key_indices:
#            continue
#        block_edges.append(
#            AttentionEdge(
#                q_idx=Q_IDX,
#                k_idx=k_idx,
#            )
#        )
#
#    return block_edges
#    

In [6]:
#with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "coincidences_sample.json")) as f:
#    coincidences = json.load(f)
#
#logger.info(f"{len(coincidences['examples'])=}")

In [7]:
#from typing import Literal
#from src.probing.prompt import prepare_probing_input
#from src.functional import predict_next_token
#from src.tokens import TokenizerOutput
#
#focus_strategy: Literal["entities", "entities_last", "ablate_all"] = "entities"
#
#effects = []
#
#for idx, c in enumerate(coincidences["examples"]):
#    entities = c["entity_pair"]
#    logger.info(f"({idx + 1}/{len(coincidences['examples'])})  {entities=}")
#
#    prompt = prepare_probing_input(
#        mt=mt,
#        entities=entities,
#        #prefix=prefix,
#        #answer_marker=answer_marker,
#        #question_marker=question_marker,
#        #block_separator=block_separator,
#        answer_prefix=" They are/were both",
#    )
#
#    logger.info(f"{prompt=}")
#
#    clean_answer = predict_next_token(
#        mt=mt, inputs=TokenizerOutput(data=prompt.tokenized), topK=1
#    )[0][0]
#    logger.info(f"{clean_answer=}")
#
#    block_edges = get_edges_to_be_ablated(
#        prompt=prompt,
#        focus_strategy=focus_strategy,
#        Q_IDX=-2,
#        whitelist_key_indices=[0]
#    )
#
#    logger.info(f"{block_edges=}")

## Patching from a different run to check contribution

In [8]:
#from src.probing.prompt import BiAssociationPrefix
#from src.utils.experiment_utils import set_seed
#
#prefix_generator_cls = BiAssociationPrefix
#
#set_seed(9001)
#
#prefix_generator = prefix_generator_cls(
#    filter_attributes=["nationality", "profession", "school"],
#    format="_3"
#)
#
#prefix = prefix_generator.get_prefix(
#    n_valid=10,
#    n_none=1
#)
#print(prefix)

In [9]:
from src.probing.prompt import BiAssociationPrefix
from src.utils.experiment_utils import set_seed
from src.probing.few_shot_examples import (
    human_nationality,
    human_profession,
    human_alma_mater
)

prefix_generator_cls = BiAssociationPrefix

prefix_generator = prefix_generator_cls(
    filter_attributes=[
        "nationality",
        "profession",
        "school"
    ],
    format = "_3"
)

set_seed(9001)
prefix = prefix_generator.get_prefix()
print(prefix)

x: 42
y: hello
# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared attribute] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person Y and Person Z
A: No - Person Y and Person Z have nothing in common.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person A and Person B
A: Yes - German - they are both German.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

## 

In [10]:
#from src.tokens import align_patching_positions, find_token_range
#
#aligned_prompts = align_patching_positions(
#    mt=mt,
#    prompt_template=sample.prompt_template,
#    clean_subj=sample.clean_entity,
#    patched_subj=sample.patched_entity,
#    trace_start_marker=prefix_generator.question_marker,
#)
#
#aligned_prompts["subj_range"]

In [11]:
import re
from typing import Optional, List
from src.dataset import ActivationPatchingSamples

def create_prompt_template(common_entity: str) -> str:
    """
    Create the prompt template with the common entity inserted.
    """
    template = f"""# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared entity] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person A and Person B
A: Yes - German - they are both German.

## Your turn, give your answer in a single line.

Q: {{}} and {common_entity}
A: Yes -"""
    
    return template

def parse_activation_data(data_string: str) -> List[ActivationPatchingSamples]:
    """
    Parse the activation patching data from the given string format.
    
    Format: common_entity | clean_entity => " clean_answer" | <-- | patched_entity => " patched_answer"
    """
    samples = []
    
    # Split by newlines and process each line
    lines = data_string.strip().split('\n')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Parse the line using regex
        # Pattern: entity1 | entity2 => " answer1" | <-- | entity3 => " answer2"
        pattern = r'^([^|]+)\|([^=]+)=> "([^"]+)" \| <-- \| ([^=]+)=> "([^"]+)"'
        match = re.match(pattern, line)
        
        if match:
            common_entity = match.group(1).strip()
            clean_entity = match.group(2).strip()
            clean_answer = match.group(3).strip()
            patched_entity = match.group(4).strip()
            patched_answer = match.group(5).strip()

            prompt_temp = create_prompt_template(common_entity)
            
            sample = ActivationPatchingSamples(
                prompt_template=prompt_temp,
                common_entity=common_entity,
                clean_entity=clean_entity,
                patched_entity=patched_entity,
                clean_answer=clean_answer,
                patched_answer=patched_answer,
                patched_answer_toks=mt.tokenizer.encode(patched_answer, add_special_tokens=False)
            )
            samples.append(sample)
        else:
            print(f"Warning: Could not parse line: {line}")
    
    return samples

data = f"""Grace Wanjiru | James Mwangi => " Ken" | <-- | Fatima Sheikh => " Urban"
Rahman Ali | Nasreen Begum => " Bang" | <-- | Takeshi Yamamoto => " Civil"
Takeshi Yamamoto | Yuki Tanaka => " Japanese" | <-- | Rahman Ali => " Civil"
Anna Schmidt | Hans Mueller => " German" | <-- | Yuki Tanaka => " Marketing"
Yuki Tanaka | Takeshi Yamamoto => " Japanese" | <-- | Anna Schmidt => " Marketing"
Ayse Kaya | Maria Santos => " Doctor" | <-- | Hans Mueller => " Economist"
Hans Mueller | Anna Schmidt => " German" | <-- | Ayse Kaya => " Economist"
Priya Patel | Rajesh Kumar => " Indian" | <-- | Sofia Hernandez => " Graphic"
Sofia Hernandez | Carlos Rodriguez => " Mexican" | <-- | Priya Patel => " Graphic"
Zahra Hosseini | Ali Rezaei => " Iranian" | <-- | Ahmed Hassan => " Pilot"
Ahmed Hassan | Layla Mahmoud => " Egyptian" | <-- | Zahra Hosseini => " Pilot"
Kwame Mensah | Rahman Ali => " Doctor" | <-- | Min-jun Park => " Ge"
Tran Thi Mai | Nguyen Van Duc => " Vietnamese" | <-- | Marco Rossi => " Architect"
Marco Rossi | Giulia Romano => " Italian" | <-- | Tran Thi Mai => " Architect"
Giulia Romano | Marco Rossi => " Italian" | <-- | Akosua Boateng => " Data"
Alexandru Popescu | Elena Ionescu => " Romanian" | <-- | David Thompson => " Environmental"
David Thompson | Sarah MacDonald => " McGill" | <-- | Alexandru Popescu => " Environmental"
António Costa | Isabel Ferreira => " Portuguese" | <-- | Jack Wilson => " Software"
Astrid Lindgren | Erik Andersson => " Swedish" | <-- | Lisa van der Berg => " Web"
Rodrigo Gonzalez | Camila Torres => " Chile" | <-- | Maria dela Rosa => " Mechanical"
"""

# Parse the data
samples = parse_activation_data(data)

# Print parsed samples
print(f"Parsed {len(samples)} samples:\n")
for i, sample in enumerate(samples[:5]):  # Show first 5
    print(f"Sample {i+1}:")
    print(f"  Prompt Template: {sample.prompt_template}")
    print(f"  Common Entity: {sample.common_entity}")
    print(f"  Clean Entity: {sample.clean_entity}")
    print(f"  Clean Answer: {sample.clean_answer}")
    print(f"  Patched Entity: {sample.patched_entity}")
    print(f"  Patched Answer: {sample.patched_answer}")
    print(f"  Patched Answer Token: {sample.patched_answer_toks}")
    print(f"  String representation: {sample}")
    print()

# Convert to JSON
print("\nJSON representation of first sample:")
print(samples[0].to_json(indent=2))

Parsed 20 samples:

Sample 1:
  Prompt Template: # Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared entity] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person A and Person B
A: Yes - German - they are both German.

## Your turn, give your answer in a single line.

Q:

In [12]:
print(str(samples[0]))

Grace Wanjiru | James Mwangi => "Ken" | <-- | Fatima Sheikh => "Urban"


In [13]:
sample_idx = 3
sample = samples[sample_idx]
print(sample)

Anna Schmidt | Hans Mueller => "German" | <-- | Yuki Tanaka => "Marketing"


In [14]:
from src.functional import generate_with_patch

mt.reset_forward()

patch_prompt = sample.prompt_template.format(sample.patched_entity)
answer = generate_with_patch(
    mt = mt,
    inputs = patch_prompt,
    do_sample=False,
    n_gen_per_prompt=1,
    remove_prefix=True
)[0]


print(f"{(sample.patched_entity, sample.common_entity)} >> {answer=}")

keywords = sample.patched_answer_toks
print(f"{[f'{t}({mt.tokenizer.decode(t)})' for t in keywords]}")

clean_prompt = sample.prompt_template.format(sample.clean_entity)
clean_answer = generate_with_patch(
    mt=mt,
    inputs=clean_prompt,
    do_sample=False,
    n_gen_per_prompt=1,
    remove_prefix=True
)[0]
print(f"{(sample.clean_entity, sample.common_entity)} >> {clean_answer=}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


('Yuki Tanaka', 'Anna Schmidt') >> answer=' Japanese - they are both Japanese.'
['68662(Marketing)']
('Hans Mueller', 'Anna Schmidt') >> clean_answer=' German - they are both German.'


In [15]:
from src.probing.prompt import BiAssociationPrefix
from src.utils.experiment_utils import set_seed
from src.probing.few_shot_examples import (
    human_nationality,
    human_profession,
    human_alma_mater,
    human_allergy,
    human_car,
    human_pet
)

prefix_generator_cls = BiAssociationPrefix

prefix_generator = prefix_generator_cls(
    filter_attributes=[
        "nationality",
        "profession",
        "school"
    ],
    format = "_3"
)

set_seed(9001)
prefix = prefix_generator.get_prefix()
print(prefix)

# Task: Find Common Attributes Between Two People
You will be given two people's names. Your job is to determine if they share ANY common attribute from the list below.

## Response Format:
- If you find a match: "Yes - [shared attribute] - [description of what they share]"
- If no match: "No - [Person 1] and [Person 2] have nothing in common"

## Attributes to Consider:
1. Same nationality → "Yes - [nationality] - they are both [nationality]"
2. Same profession → "Yes - [profession] - they are both [profession]"
3. Same school → "Yes - [school] - they both graduated from [school]"

Q: Person Y and Person Z
A: No - Person Y and Person Z have nothing in common.

Q: Person C and Person D
A: Yes - Doctor - they are both doctors.

Q: Person W and Person X
A: No - Person W and Person X have nothing in common.

Q: Person A and Person B
A: Yes - German - they are both German.

Q: Person E and Person F
A: Yes - Boston University - they both graduated from Boston University.

## Your turn, give

In [16]:
from src.tokens import align_patching_positions

aligned_prompts = align_patching_positions(
    mt=mt,
    prompt_template=sample.prompt_template,
    clean_subj=sample.clean_entity,
    patched_subj=sample.patched_entity,
    trace_start_marker=prefix_generator.question_marker
)

aligned_prompts["subj_range"]

trace_start_idx=245


(245, 249)

In [28]:
from src.tokens import prepare_input, find_token_range
from src.utils.typing import TokenizerOutput

text = mt.tokenizer.decode(
    aligned_prompts["clean_input"]["input_ids"][0], skip_special_tokens=False
)

clean_inputs = prepare_input(
    prompts=text, tokenizer=mt, add_special_tokens=False, return_offsets_mapping=True
)

assert torch.allclose(
    clean_inputs.input_ids, aligned_prompts["clean_input"]["input_ids"]
)

print(f"{sample.common_entity=}")

common_entity_range = find_token_range(
    string=text,
    substring=sample.common_entity,
    tokenizer=mt,
    offset_mapping=clean_inputs.offset_mapping[0],
    add_special_tokens=False,
)

print(f"{common_entity_range=}")

tokenized_input = TokenizerOutput(data=aligned_prompts["clean_input"])
print(
    f'"{mt.tokenizer.decode(tokenized_input.input_ids[0][common_entity_range[0] : common_entity_range[1]])}"'
)

sample.common_entity='Anna Schmidt'
common_entity_range=(250, 252)
" Anna Schmidt"


In [33]:
from src.probing.utils import ProbingPrompt

aligned_clean_prompt = ProbingPrompt(
    prompt=mt.tokenizer.decode(
        aligned_prompts["clean_input"]["input_ids"][0], skip_special_tokens=False
    ),
    entities=[sample.clean_entity, sample.common_entity],
    model_key=model_key,
    tokenized=aligned_prompts["clean_input"],
    entity_ranges=[
        aligned_prompts["subj_range"],
        common_entity_range,
    ],
    query_range=[-1, -1]
)

print(
    f'{sample.clean_entity=} | "{mt.tokenizer.decode(aligned_clean_prompt.tokenized.input_ids[0][aligned_prompts["subj_range"][0] : aligned_prompts["subj_range"][1]])}"'
)
print(
    f'{sample.common_entity=} | "{mt.tokenizer.decode(aligned_clean_prompt.tokenized.input_ids[0][common_entity_range[0] : common_entity_range[1]])}"'
)

aligned_patch_prompt = ProbingPrompt(
    prompt=mt.tokenizer.decode(
        aligned_prompts["patched_input"]["input_ids"][0], skip_special_tokens=False
    ),
    entities=[sample.patched_entity, sample.common_entity],
    model_key=model_key,
    tokenized=aligned_prompts["patched_input"],
    entity_ranges=[
        aligned_prompts["subj_range"],
        common_entity_range,
    ],
    query_range=[-1, -1],
)

print(
    f'{sample.patched_entity=} | "{mt.tokenizer.decode(aligned_patch_prompt.tokenized.input_ids[0][aligned_prompts["subj_range"][0] : aligned_prompts["subj_range"][1]])}'
)
print(
    f'{sample.common_entity=} | "{mt.tokenizer.decode(aligned_patch_prompt.tokenized.input_ids[0][common_entity_range[0] : common_entity_range[1]])}"'
)

sample.clean_entity='Hans Mueller' | "<|eot_id|><|eot_id|> Hans Mueller"
sample.common_entity='Anna Schmidt' | " Anna Schmidt"
sample.patched_entity='Yuki Tanaka' | " Yuki Tanaka
sample.common_entity='Anna Schmidt' | " Anna Schmidt"
